<a href="https://colab.research.google.com/github/anissval/oneAPI/blob/main/Intel%20AI%20Analytics%20Toolkit/random_forest_yolanda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intel® Extension for Scikit-learn Random Forest for Yolanda dataset

In [10]:
! python -m pip install --upgrade pip
! python -m pip install scikit-learn-intelex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for scikit-learn-intelex from https://files.pythonhosted.org/packages/3d/89/36bf3b024b86ae3c1de86fcaa11ef6e6dfc4d223f491cc648ce6edeea74f/scikit_learn_intelex-2023.2.1-py310-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for daal4py==2023.2.1 from https://files.pythonhosted.org/packages/18/7a/a3ecef294e068a38d8e8d370e84127afca3f95d726d24b044e8928aa1cf8/daal4py-2023.2.1-py310-none-manylinux1_x86_64.whl.metadata
  Obtaining dependency information for daal==2023.2.1 from https://files.pythonhosted.org/packages/e0/51/9369c49b50a51279660c31b0ec22afcb649f082fd2753915d1306dc6d038/daal-2023.2.1-py2.py3-none-manylinux1_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 6.9 MB/s eta 0:00:00
  

# Resolution for Known Issue on Google Colab

In [11]:
import sys
import os
import site

sys.path.append(os.path.join(os.path.dirname(site.getsitepackages()[0]), "site-packages"))

In [12]:
print(os.path.join(os.path.dirname(site.getsitepackages()[0]), "site-packages"))

/usr/local/lib/python3.10/site-packages


In [13]:
print(os.path.dirname(site.getsitepackages()[0]))

/usr/local/lib/python3.10


In [14]:
print(sys.path.append(os.path.join(os.path.dirname(site.getsitepackages()[0]), "site-packages")))

None


In [15]:
from timeit import default_timer as timer
from sklearn import metrics
from IPython.display import HTML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [16]:
!python --version

Python 3.10.12


### Download the data

In [17]:
x, y = fetch_openml(name="Yolanda", return_X_y=True)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Split the data into train and test sets

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=72)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((280000, 100), (120000, 100), (280000,), (120000,))

### Patch original Scikit-learn with Intel® Extension for Scikit-learn
Intel® Extension for Scikit-learn (previously known as daal4py) contains drop-in replacement functionality for the stock Scikit-learn package. You can take advantage of the performance optimizations of Intel® Extension for Scikit-learn by adding just two lines of code before the usual Scikit-learn imports:

In [19]:
from sklearnex import patch_sklearn

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Intel® Extension for Scikit-learn patching affects performance of specific Scikit-learn functionality. Refer to the [list of supported algorithms and parameters](https://intel.github.io/scikit-learn-intelex/algorithms.html) for details. In cases when unsupported parameters are used, the package fallbacks into original Scikit-learn. If the patching does not cover your scenarios, [submit an issue on GitHub](https://github.com/intel/scikit-learn-intelex/issues).

Training Random Forest algorithm with Intel® Extension for Scikit-learn for Yolanda dataset

In [20]:
from sklearn.ensemble import RandomForestRegressor

params = {"n_estimators": 150, "random_state": 44, "n_jobs": -1}
start = timer()
rf = RandomForestRegressor(**params).fit(x_train, y_train)
train_patched = timer() - start
f"Intel® extension for Scikit-learn time: {train_patched:.2f} s"

'Intel® extension for Scikit-learn time: 2081.74 s'

Predict and get a result of the Random Forest algorithm with Intel® Extension for Scikit-learn

In [21]:
y_pred = rf.predict(x_test)
mse_opt = metrics.mean_squared_error(y_test, y_pred)
f"Intel® extension for Scikit-learn Mean Squared Error: {mse_opt}"

'Intel® extension for Scikit-learn Mean Squared Error: 83.60018104634611'

### Train the same algorithm with original Scikit-learn
In order to cancel optimizations, we use *unpatch_sklearn* and reimport the class RandomForestRegressor.

In [22]:
from sklearnex import unpatch_sklearn

unpatch_sklearn()

Training Random Forest algorithm with original Scikit-learn library for Yolanda dataset

In [23]:
from sklearn.ensemble import RandomForestRegressor

start = timer()
rf = RandomForestRegressor(**params).fit(x_train, y_train)
train_unpatched = timer() - start
f"Original Scikit-learn time: {train_unpatched:.2f} s"

'Original Scikit-learn time: 4453.85 s'

Predict and get a result of the Random Forest algorithm with original Scikit-learn

In [24]:
y_pred = rf.predict(x_test)
mse_original = metrics.mean_squared_error(y_test, y_pred)
f"Original Scikit-learn Mean Squared Error: {mse_opt}"

'Original Scikit-learn Mean Squared Error: 83.60018104634611'

In [25]:
HTML(
    f"<h3>Compare MSE metric of patched Scikit-learn and original</h3>"
    f"MSE metric of patched Scikit-learn: {mse_opt} <br>"
    f"MSE metric of unpatched Scikit-learn: {mse_original} <br>"
    f"Metrics ratio: {mse_opt/mse_original} <br>"
    f"<h3>With Scikit-learn-intelex patching you can:</h3>"
    f"<ul>"
    f"<li>Use your Scikit-learn code for training and prediction with minimal changes (a couple of lines of code);</li>"
    f"<li>Fast execution training and prediction of Scikit-learn models;</li>"
    f"<li>Get the similar quality</li>"
    f"<li>Get speedup in <strong>{(train_unpatched/train_patched):.1f}</strong> times.</li>"
    f"</ul>"
)